# 基本设置

In [1]:
# encoding=utf-8

# X_train, X_test, y_train, y_test

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif

import pickle

import numpy as np
from sklearn import cross_validation
from sklearn.cross_validation import KFold  
from sklearn.linear_model import LogisticRegression  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn import svm  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier  

from sklearn.datasets.base import Bunch
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import ExtraTreesClassifier

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import joblib
from sklearn import metrics
%matplotlib inline

D:\software\conda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    
    def transform(self, X):
        return [[len(x),self.getcnt(x),self.getcnt(x)/len(x),self.getnegcnt(x),self.getnegcnt(x)/len(x)] for x in X]
    

In [3]:
def classified_index(corpus_p, corpus_n, corpus, lab):
    '''
    二分类模型各指标的结果
    '''
    
    res = chapter_pipeline_1.predict(corpus_p)
    print('正样本预测准确率: ', float(sum(res))/len(res),len(res))
    
    res = chapter_pipeline_1.predict(corpus_n)
    print('负样本预测准确率: ',  1-float(sum(res))/len(res),len(res))
    
    y_pred_class = chapter_pipeline_1.predict(corpus)
    print('accuracy_score: ', metrics.accuracy_score(lab, y_pred_class)) # 指所有分类正确的百分比
    print(metrics.classification_report(lab, y_pred_class))
    print('confusion_matrix: ')
    print( metrics.confusion_matrix(lab, y_pred_class))

# 银监会--倾向性模型

## 导入预处理后正负预测样本

In [64]:
# pos samples
corpus_p = []   
lab_p = [] 
f = open("0609_p_1.txt","r+", encoding='UTF-8')
for content in f:
    corpus_p.append(content.split("\t")[0])
    lab_p.append(1)
f.close()  

# neg samples
corpus_n = []   
lab_n = []   
f = open("0609_n_1.txt","r+", encoding='UTF-8')
for content in f:
    corpus_n.append(content.split("\t")[0])
    lab_n.append(0)
f.close()

corpus, lab = [], []
corpus = corpus_p + corpus_n
lab = lab_p + lab_n

# 导入模型
chapter_pipeline_1 = joblib.load("NP_model_b.pkl.z")  

# 结果查看
classified_index(corpus_p, corpus_n, corpus, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.982 500


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.864321608040201 398
accuracy_score:  0.9298440979955457
             precision    recall  f1-score   support

          0       0.97      0.86      0.92       398
          1       0.90      0.98      0.94       500

avg / total       0.93      0.93      0.93       898

confusion_matrix: 
[[344  54]
 [  9 491]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## 导入原始数据
- 0604-0608银监会负面
- 0604-0608银监误判为负面的数据

In [39]:
# pos samples
corpus_p = []   
lab_p = [] 
cbrc_pos_filename = 'data/0609_pos_raw_data.txt'
f = open(cbrc_pos_filename,"r+", encoding='UTF-8')
for content in f:
    corpus_p.append(content.split("\t")[0])
    lab_p.append(1)
f.close()  

# neg samples
corpus_n = []   
lab_n = []   
cbrc_neg_filename = 'data/0609_neg_raw_data.txt'
f = open(cbrc_neg_filename,"r+", encoding='UTF-8')
for content in f:
    corpus_n.append(content.split("\t")[0])
    lab_n.append(0)
f.close()

corpus, lab = [], []
corpus = corpus_p + corpus_n
lab = lab_p + lab_n

# 预处理
import pre
corpus_p = pre.handle_contents(corpus_p)
corpus_n = pre.handle_contents(corpus_n)
corpus = pre.handle_contents(corpus)
del pre

# 导入模型
chapter_pipeline_1 = joblib.load("NP_model_b.pkl.z")  

# 查看结果
classified_index(corpus_p, corpus_n, corpus, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.97 500


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.864321608040201 398
accuracy_score:  0.923162583518931
             precision    recall  f1-score   support

          0       0.96      0.86      0.91       398
          1       0.90      0.97      0.93       500

avg / total       0.93      0.92      0.92       898

confusion_matrix: 
[[344  54]
 [ 15 485]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [27]:
len(corpus)

898

## 银监会：结论
- 韩老师预处理后数据，使用模型预测后结果：0.983/0.86/0.93（正样本精度、负样本精度、总体正确率）
- 线上程序预处理后数据，使用模型预测和结果：0.975/0.86/0.92（正样本精度、负样本精度、总体正确率）
- 两套程序基本一致

# 保监会--倾向性模型

## 导入原始数据
- 0604-0608保监会非负补充
- 0604-0608保监会负面
- 0604-0608保监会误判为负面的数据

In [4]:
# pos samples
corpus_p = []   
lab_p = [] 
circ_pos_filename = 'data/circ_0609_pos_raw_data.txt'
f = open(circ_pos_filename,"r+", encoding='UTF-8')
for content in f:
    corpus_p.append(content.split("\t")[0])
    lab_p.append(1)
f.close()  

# neg samples
corpus_n = []   
lab_n = []   
circ_neg_filename = 'data/circ_0609_neg_raw_data.txt'
f = open(circ_neg_filename,"r+", encoding='UTF-8')
for content in f:
    corpus_n.append(content.split("\t")[0])
    lab_n.append(0)
f.close()

corpus, lab = [], []
corpus = corpus_p + corpus_n
lab = lab_p + lab_n

In [5]:
print(len(corpus))
print(len(corpus_p))
print(len(corpus_n))

767
399
368


### 预处理（true） + 模型（NP_model_i）

In [58]:
# 预处理  True
import pre
corpus_p_i = pre.handle_contents(corpus_p)
corpus_n_i = pre.handle_contents(corpus_n)
corpus_i = pre.handle_contents(corpus)

# 导入模型
chapter_pipeline_1 = joblib.load("NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.6616541353383458 399


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.8206521739130435 368
accuracy_score:  0.7379400260756193
             precision    recall  f1-score   support

          0       0.69      0.82      0.75       368
          1       0.80      0.66      0.72       399

avg / total       0.75      0.74      0.74       767

confusion_matrix: 
[[302  66]
 [135 264]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### 预处理（true） + 模型（0609_NP_model_i）

In [59]:
# 预处理  True
import pre
corpus_p_i = pre.handle_contents(corpus_p)
corpus_n_i = pre.handle_contents(corpus_n)
corpus_i = pre.handle_contents(corpus)

# 导入模型
chapter_pipeline_1 = joblib.load("0609_NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.9899749373433584 399


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.016304347826086918 368
accuracy_score:  0.5228161668839635
             precision    recall  f1-score   support

          0       0.60      0.02      0.03       368
          1       0.52      0.99      0.68       399

avg / total       0.56      0.52      0.37       767

confusion_matrix: 
[[  6 362]
 [  4 395]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### 预处理（false） + 模型（NP_model_i）

In [60]:
# 预处理  False
import pre_1
corpus_p_i = pre_1.handle_contents(corpus_p)
corpus_n_i = pre_1.handle_contents(corpus_n)
corpus_i = pre_1.handle_contents(corpus)

# 导入模型
chapter_pipeline_1 = joblib.load("NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.6616541353383458 399


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.8206521739130435 368
accuracy_score:  0.7379400260756193
             precision    recall  f1-score   support

          0       0.69      0.82      0.75       368
          1       0.80      0.66      0.72       399

avg / total       0.75      0.74      0.74       767

confusion_matrix: 
[[302  66]
 [135 264]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### 预处理（false） + 模型（0609_NP_model_i）

In [61]:
# 预处理  False
import pre_1
corpus_p_i = pre_1.handle_contents(corpus_p)
corpus_n_i = pre_1.handle_contents(corpus_n)
corpus_i = pre_1.handle_contents(corpus)

# 导入模型
chapter_pipeline_1 = joblib.load("0609_NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.9899749373433584 399


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.016304347826086918 368
accuracy_score:  0.5228161668839635
             precision    recall  f1-score   support

          0       0.60      0.02      0.03       368
          1       0.52      0.99      0.68       399

avg / total       0.56      0.52      0.37       767

confusion_matrix: 
[[  6 362]
 [  4 395]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### 预处理（false 韩） + 模型（NP_model_i）

In [63]:
# 预处理  False
import pre_2
corpus_p_i = pre_2.handle_contents(corpus_p)
corpus_n_i = pre_2.handle_contents(corpus_n)
corpus_i = pre_2.handle_contents(corpus)

# 导入模型
chapter_pipeline_1 = joblib.load("NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.6641604010025063 399


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.8206521739130435 368
accuracy_score:  0.7392438070404173
             precision    recall  f1-score   support

          0       0.69      0.82      0.75       368
          1       0.80      0.66      0.73       399

avg / total       0.75      0.74      0.74       767

confusion_matrix: 
[[302  66]
 [134 265]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### 预处理（false 韩） + 模型（0609_NP_model_i）

In [9]:
# 预处理  False
import pre_2
corpus_p_i = pre_2.handle_contents(corpus_p)
corpus_n_i = pre_2.handle_contents(corpus_n)
corpus_i = pre_2.handle_contents(corpus)

# 导入模型
from sklearn.externals import joblib
chapter_pipeline_1 = joblib.load("0613_NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.8170426065162907 399


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.7934782608695652 368
accuracy_score:  0.8057366362451108
             precision    recall  f1-score   support

          0       0.80      0.79      0.80       368
          1       0.81      0.82      0.81       399

avg / total       0.81      0.81      0.81       767

confusion_matrix: 
[[292  76]
 [ 73 326]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
corpus = []
lab = []
corpus_p_i = []
lab_p_i = []
corpus_n_i = []
lab_n_i = []

f = open("0613_p.txt","r+", encoding='UTF-8')
for content in f:
    corpus_p_i.append(content.split("\t")[0])
    lab_p_i.append(1)
f.close()

#neg sample
f = open("0613_n.txt","r+", encoding='UTF-8')
for content in f:
    corpus_n_i.append(content.split("\t")[0])
    lab_n_i.append(0)
f.close()

corpus_i = corpus_p_i + corpus_n_i
lab = lab_p_i + lab_n_i

# 导入模型
from sklearn.externals import joblib
chapter_pipeline_1 = joblib.load("0613_NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.8220551378446115 399


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.8043478260869565 368
accuracy_score:  0.8135593220338984
             precision    recall  f1-score   support

          0       0.81      0.80      0.81       368
          1       0.82      0.82      0.82       399

avg / total       0.81      0.81      0.81       767

confusion_matrix: 
[[296  72]
 [ 71 328]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
corpus = []
lab = []
corpus_p_i = []
lab_p_i = []
corpus_n_i = []
lab_n_i = []

f = open("0609_p_1.txt","r+", encoding='UTF-8')
for content in f:
    corpus_p_i.append(content.split("\t")[0])
    lab_p_i.append(1)
f.close()
f = open("0609_p_2.txt","r+", encoding='UTF-8')
for content in f:
    corpus_p_i.append(content.split("\t")[0])
    lab_p_i.append(1)
f.close()

#neg sample
f = open("0609_n_1.txt","r+", encoding='UTF-8')
for content in f:
    corpus_n_i.append(content.split("\t")[0])
    lab_n_i.append(0)
f.close()

corpus_i = corpus_p_i + corpus_n_i
lab = lab_p_i + lab_n_i

# 导入模型
from sklearn.externals import joblib
chapter_pipeline_1 = joblib.load("0613_NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.9933774834437086 755
负样本预测准确率:  0.9899497487437185 398
accuracy_score:  0.9921942758022549
             precision    recall  f1-score   support

          0       0.99      0.99      0.99       398
          1       0.99      0.99      0.99       755

avg / total       0.99      0.99      0.99      1153

confusion_matrix: 
[[394   4]
 [  5 750]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
# 导入模型
import joblib
chapter_pipeline_1 = joblib.load("NP_model_i.pkl.z")  

# 查看结果
classified_index(corpus_p_i, corpus_n_i, corpus_i, lab)

D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


正样本预测准确率:  0.9496688741721855 755


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


负样本预测准确率:  0.7412060301507537 398
accuracy_score:  0.8777103209019947
             precision    recall  f1-score   support

          0       0.89      0.74      0.81       398
          1       0.87      0.95      0.91       755

avg / total       0.88      0.88      0.87      1153

confusion_matrix: 
[[295 103]
 [ 38 717]]


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## 保监会：结果 + 结论
> - 预处理（true）：预处理时 jieba 分词使用模式 cut_all=True
> - 模型（NP_model_i）：预测所用模型
> - 韩：韩老师所用预处理程序

- 预处理（true） + 模型（NP_model_i）：0.66、0.82、0.7379（正样本精度、负样本精度、总体正确率）
- 预处理（true） + 模型（0609_NP_model_i）：0.98997/0.016/0.52
- 预处理（false） + 模型（NP_model_i）：0.66、0.82、0.7379
- 预处理（false） + 模型（0609_NP_model_i）：0.98997/0.016/0.52 
- 预处理（false 韩） + 模型（NP_model_i）：0.66/0.82/0.7392
- 预处理（false 韩） + 模型（0609_NP_model_i）：0.98997/0.016/0.52 

- 预处理过程中的分词模式影响不大。（周一中 cut_all=True 设置的影响没有重复出来）
- 新旧模型在精准率（precision）上的差别较小，而在召回率上差异较大。
- 新模型对正样本的 召回率 超高（0.99），对负样本的 召回率 超低（0.02）。说明标识为正样本的数量实际为正样本的比例高，而标识为负样本实际为负样本的比例很低。